<a href="https://colab.research.google.com/github/DigiClau/stablediffusion_webui/blob/main/ComfyUI_digiclau.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 이 Google Colab notebook은 [ComfyUI](https://github.com/comfyanonymous/ComfyUI)를 실행해 줍니다.

## ❗❗❗ [ComfyUI Google Colab 사용 설명 영상](https://youtu.be/HUY2WdbNQmc) - 자세한 사용 설명을 보실수 있습니다.
### ❗❗❗ 이 Colab notebook을 사용하기 전에, 바로 위에 있는 사용 설명 영상을 먼저 시청하시기 바랍니다.

### [DigiClau Youtube](https://www.youtube.com/channel/UCllY989CI5F-eU1BJ-rwEvw) - 저의 채널도 방문해 주세요.

### 복사본을 사용할 경우, 원본은 [이곳](https://colab.research.google.com/github/DigiClau/stablediffusion_webui/blob/main/ComfyUI_digiclau.ipynb)에 있습니다.

### 🍷🍷🍷 [SD Automatic1111 Colab도 있습니다.](https://colab.research.google.com/github/DigiClau/stablediffusion_webui/blob/main/StableDiffusionWebUI_digiclau.ipynb) 🍷🍷🍷
### 🍷🍷🍷 [Stable Diffusion WebUI Forge Colab도 있습니다.](https://colab.research.google.com/github/DigiClau/stablediffusion_webui/blob/main/StableDiffusionWebUI_digiclau_Forge.ipynb) 🍷🍷🍷

## [☕커피 사주세요](https://www.buymeacoffee.com/digiclau)

## ### 최근 업데이트 정보
- 2024년 6월:
 - ComfyUI에 필요한 Pytorch 및 dependency 업데이트


In [ ]:
#@title <font size="6" color="black">ComfyUI 시작하는 섹션</font>

from pathlib import Path

OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
USE_COMFYUI_MANAGER = True  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
OPTIONS['USE_COMFYUI_MANAGER'] = USE_COMFYUI_MANAGER
OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES'] = INSTALL_CUSTOM_NODES_DEPENDENCIES

current_dir = !pwd
WORKSPACE = f"{current_dir[0]}/ComfyUI"

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull

!echo -= Install dependencies =-
!pip3 install accelerate
!pip3 install einops transformers>=4.25.1 safetensors>=0.3.0 aiohttp pyyaml Pillow scipy tqdm psutil
# 2024.06.07 pytorch 2.1.0 -> 2.3.0으로 업뎃
!pip3 install xformers!=0.0.18 torch>=2.3.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip3 install torchsde

# 2024.06.07 필요한 요구사항 업뎃 설치
!pip install -r requirements.txt

if OPTIONS['USE_COMFYUI_MANAGER']:
  %cd custom_nodes
  ![ ! -d ComfyUI-Manager ] && echo -= Initial setup ComfyUI-Manager =- && git clone https://github.com/ltdrdata/ComfyUI-Manager
  %cd ComfyUI-Manager
  !git pull

%cd $WORKSPACE

if OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES']:
  !pwd
  !echo -= Install custom nodes dependencies =-
  ![ -f "custom_nodes/ComfyUI-Manager/scripts/colab-dependencies.py" ] && python "custom_nodes/ComfyUI-Manager/scripts/colab-dependencies.py"
print("\nAll done\n")

##################################################
# Run ComfyUI with cloudflared (Recommended Way) #
##################################################
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')

threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()
!python main.py --dont-print-server
